In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from tensorflow.python.keras.backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import csv
import random
import itertools
import time
import pickle
from data_utils import *

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Permute, Reshape
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
from tensorflow.keras.losses import MSE

## Import Dataset

In [ ]:
sampling_rate = 100
window_size = 550
stride = 50
f = '/home/nesl/209as_sec/human_act/Data/Activity_Dataset/'

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300,channels=0)

X_tr = X_tr.swapaxes(1,2)
X_test = X_test.swapaxes(1,2)



_, dim, win_len = X_tr.shape

X_tr = np.reshape(X_tr, (-1, dim, win_len, 1))
X_test = np.reshape(X_test, (-1, dim, win_len, 1))

print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

## Load Model

In [ ]:
model = load_model('/home/nesl/209as_sec/human_act/Big Models/CNN/cnn.h5')
model.summary()

## Attack

In [ ]:
def fgsm_attack(model, image, label, eps):
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(image)
        pred = model(image)
        loss = MSE(label, pred)
        gradient = tape.gradient(loss, image)
        signedGrad = tf.sign(gradient)
        adversary = (image + (signedGrad * eps)).numpy()
        return adversary

In [ ]:
eps = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.5,2.0,5.0,10.0,15.0,20.0,30.0,40.0,50.0]
accu_num = []
for item in eps:
    countadv = 0
    for i in tqdm(range(len(X_test))):
        act = X_test[i,:,:].reshape(1,6,550,1)
        label = Y_test[i,:]
        actPred = model.predict(act)
        actPred = actPred.argmax()
        adversary = fgsm_attack(model,act, label, eps=item)
        pred = model.predict(adversary)
        adversaryPred = pred[0].argmax()
        if actPred == adversaryPred:
            countadv += 1
        
 
    print("Adversarial accuracy : ", countadv / len(X_test))
    accu_num.append(countadv / len(X_test))       

In [ ]:
accu_num